In [1]:
import torch

/Users/sofija/miniforge3/envs/torchgeo_dev/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import tempfile
from glob import glob
from typing import Any, Callable, Dict, Optional, Iterator, Union, Tuple, List
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torch

from torch.utils.data import DataLoader
from torchgeo.datasets import BoundingBox, stack_samples

import torch.nn.functional as F
from torchgeo.samplers.batch import RandomBatchGeoSampler
from torchgeo.samplers.single import GridGeoSampler, RandomGeoSampler
from torchgeo.samplers.utils import get_random_bounding_box
from torchgeo.samplers.constants import Units

import math
import random
import numpy as np

In [3]:
os.chdir("/Users/sofija/Ai4er/gtc/WildfireDistribution/")

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from src.data_loading import LandcoverSimple, LandcoverComplex, MODIS_CCI, MODIS_JD, Landsat7
from src.samplers import ConstrainedRandomBatchGeoSampler
from src.datamodules import MODISJDLandcoverSimpleDataModule , MODISJDLandcoverSimpleLandsatDataModule

In [ ]:
land = LandcoverSimple()

In [6]:
batch_size = 2

In [7]:
sampler_size = 256

In [8]:
length = 10

In [17]:
datamodule = MODISJDLandcoverSimpleDataModule(
    modis_root_dir="data/modis/2017/",
    landcover_root_dir="data/landcover/Classified/",
    patch_size=sampler_size,
    length=length,
    batch_size=batch_size,
    num_workers=2,
    one_hot_encode=False,
    balance_samples=False,
    grid_sampler=False,
)

datamodule_balanced = MODISJDLandcoverSimpleDataModule(
    modis_root_dir="data/modis/2017/",
    landcover_root_dir="data/landcover/Classified/",
    patch_size=sampler_size,
    batch_size=batch_size,
    length=length,
    num_workers=0,
    one_hot_encode=False,
    balance_samples=True,
    burn_prop = 0.5,
)

In [22]:
from torchgeo.trainers import SemanticSegmentationTask
from src.tasks import BinarySemanticSegmentationTask

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from pytorch_lightning.callbacks import Callback

In [23]:
# model = BinarySemanticSegmentationTask(
#      segmentation_model="unet",
#      encoder_name="resnet18",
#      encoder_weights="imagenet",
#      in_channels=1,
#      num_filters=32,
#      num_classes=2,
#      loss="jaccard",
#      learning_rate=0.1,
#      ignore_zeros=False,
#      learning_rate_schedule_patience=5,
#  )


model = SemanticSegmentationTask(
     segmentation_model="unet",
     encoder_name="resnet18",
     encoder_weights="imagenet",
     in_channels=1,
     num_filters=32,
     num_classes=2,
     loss="jaccard",
     learning_rate=0.1,
     ignore_zeros=False,
     learning_rate_schedule_patience=5,
 )

In [24]:
trainer = Trainer(fast_dev_run=True,
               )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).


In [25]:
trainer.fit(model=model, datamodule=datamodule)

/Users/sofija/miniforge3/envs/torchgeo_dev/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | Unet             | 14.3 M
1 | loss          | JaccardLoss      | 0     
2 | train_metrics | MetricCollection | 0     
3 | val_metrics   | MetricCollection | 0     
4 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.288    Total estimated model params size (MB)


Epoch 0:   0%|                                                | 0/2 [00:00<?, ?it/s]

ValueError: ctypes objects containing pointers cannot be pickled